In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\user\\Desktop\\Repositories\\Natural-Language-Processing\\Medical Advice Chatbot'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents



In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob= "*.pdf", loader_cls=PyPDFLoader)

    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("Data/")

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20 )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5860

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [11]:
embeddings = download_hugging_face_embeddings()


C:\Users\user\AppData\Local\Temp\ipykernel_21876\826149639.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


In [12]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

384

In [23]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv("PINE_CONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [16]:
from pinecone import Pinecone, ServerlessSpec

index_name = "medibot"
pc = Pinecone(api_key=PINECONE_API_KEY)
pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [18]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
documents=text_chunks,
index_name=index_name,
embedding=embeddings,
)


In [19]:
# Load Existing Index
docsearch = PineconeVectorStore.from_existing_index(
index_name=index_name,
embedding=embeddings,
)

In [20]:
retriver = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k": 4})

In [22]:
retrived_docs = retriver.invoke("What is Acne")
retrived_docs

[Document(id='fe292c7a-1038-4e8c-bf65-ac79308b1e2a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical Book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='4a0c292d-ef42-4669-800f-4b188c40c7d5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical Book.pdf', 'total_pages': 637.0}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteri

In [24]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain. chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. ",
    "Use the following pieces of retrieved context to answer ",
    "the question. If you don't know the answer, say that you ",
    "don't know. Use three sentences maximum and keep the ",
    "answer concise.",
    "\n\n",
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


ValueError: Invalid template: ('You are an assistant for question-answering tasks. ', 'Use the following pieces of retrieved context to answer ', "the question. If you don't know the answer, say that you ", "don't know. Use three sentences maximum and keep the ", 'answer concise.', '\n\n', '[{context}]')